In [1]:
import syft as sy
import numpy as np
import pandas as pd

from syft.core.adp.data_subject import DataSubject

/opt/anaconda3/envs/syft_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/CarlaFC/Star-Wars/main/characters.csv')
df.columns

Index(['name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color',
       'birth_year', 'gender', 'homeworld', 'species'],
      dtype='object')

In [3]:
# how many total characters?

In [4]:
data_subjects = []
for n in df.name.tolist():
    data_subjects.append(DataSubject(name=n))

In [5]:
data_subjects

[<DataSubject:Luke Skywalker>,
 <DataSubject:C-3PO>,
 <DataSubject:R2-D2>,
 <DataSubject:Darth Vader>,
 <DataSubject:Leia Organa>,
 <DataSubject:Owen Lars>,
 <DataSubject:Beru Whitesun lars>,
 <DataSubject:R5-D4>,
 <DataSubject:Biggs Darklighter>,
 <DataSubject:Obi-Wan Kenobi>,
 <DataSubject:Anakin Skywalker>,
 <DataSubject:Wilhuff Tarkin>,
 <DataSubject:Chewbacca>,
 <DataSubject:Han Solo>,
 <DataSubject:Greedo>,
 <DataSubject:Jabba Desilijic Tiure>,
 <DataSubject:Wedge Antilles>,
 <DataSubject:Jek Tono Porkins>,
 <DataSubject:Yoda>,
 <DataSubject:Palpatine>,
 <DataSubject:Boba Fett>,
 <DataSubject:IG-88>,
 <DataSubject:Bossk>,
 <DataSubject:Lando Calrissian>,
 <DataSubject:Lobot>,
 <DataSubject:Ackbar>,
 <DataSubject:Mon Mothma>,
 <DataSubject:Arvel Crynyd>,
 <DataSubject:Wicket Systri Warrick>,
 <DataSubject:Nien Nunb>,
 <DataSubject:Qui-Gon Jinn>,
 <DataSubject:Nute Gunray>,
 <DataSubject:Finis Valorum>,
 <DataSubject:Jar Jar Binks>,
 <DataSubject:Roos Tarpals>,
 <DataSubject:Rugor 